*Report Basics of Mobile Robotics*  
## Students: 
- Levy Alexandre 
- Tourki Amine 
- Tourki Emna 
- Zghari Aya






Link to our [Github](https://github.com/AmineTourki/Mobile-Robotics-)

## Imports

In [ ]:
import numpy as np
import math as m
import cv2
import os
import sys
import matplotlib.pyplot as plt
import time
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))
from A_star_algorithm import run_A_Star
from vision_agent import Vision
import local_navigation
import navigation
import utils
import  Kalman

%matplotlib inline



**Introduction**  
This project aims to make a Thymio navigate in a course where it will have to use Computer Vision, Global and Local Navigation with a filter to reduce positioning errors.

The arena that we present you is modular, the obstacles can change place and the start and goal can be completely different. In this arena we also force our robot to pass through a checkpoint, which will spice up the route it has to make.

## Method Overview



The camera computes a global map from images captured from the webcam. Global navigation works with the A* algorithm. The Thymio first goes to the checkpoint in green and then to the final point in red.  If unexpected obstacles are encountered on the path of the THymio, local avoidance is triggered to avoid the obstacle ahead with proximity sensors and a replanning is sent. A kalman filter is used to reduce the noise and have a perfect trajectory. 

<img src="Images/Schema_project.png" width="600"/>

## Vision

## Map Setup

<img src="Images/Setup.png" width="600"/>

## Implementation details

The vision Class is strongly dependent on the map data, which can be updated continuously from the video. The construction of a global map consists of the following steps:

1. The raw images are sampled at a given frequency from the webcam.
2. A rebounding of the arena is done.
3. Multiple color masks are applied to the transformed image to distinguish the objectives,the thymio, the arena and obstacles.
4. Multiple color masks are applied to the transformed image to distinguish the target position, thymio (front and rear) and obstacles.

Note:
The rebounding and the localisation of the objectives and the obstacle are done on the first image. The next images are used for the robot position given to the Kalman filter.   
The open library CV2 was used for the vision module.


- Input
    
    - Raw image captured from camera
    - Distance in real between the two circles on the robot to have a real world scale
    - Color HSV values that helps define the map (this is sensitive to light and needs to be calibrated after setting environment)
    
- Output
    - Global map that marks all occupancy with 1 and vacanncy with 0
    - Starting pose, checkpoint pose and finish pose of the Thymio

- Limitations 
    - The color masks are rather sensitive to lighting conditions and hyperparameters had to be calibrated
    
- Keyparameters
    - Color mask Table
        | Color | Used for | Lower Bounds | Upper Bounds |
        | --- | --- | --- | --- |
        | Blue | Robot Position | [87,97,50] | [164,255,160] |
        | Red | Final Destination | [150,70,50] | [190,255,255] |
        | Green | Checkpoint | [40,40,40] | [100,255,255] |
        | Black | Robot Position | [0,0,0] | [180,255,50] |
        | White+Green+Blue+Red | Reboarder Map |  [0,0,40] | [255,255,255] |



### Class Vision

Here is the class that can be found in vision_agent.py and its initialization:

    class Vision_Agent:
    def __init__(self):
        self.converter = 0
        self.cam = cv2.VideoCapture(1)
        ret, frame = self.cam.read()  # initialize
        self.image = frame
        self.resize()                 #resize the image
        self.hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV) # convert to HSV
        self.get_first_image_cam(self.image)                   # delimit of the image
        self.hsv = cv2.cvtColor(self.image, cv2.COLOR_BGR2HSV)
        
        
        self.angle = 0              # angle of the robot
        self.center_robot = [0,0]   # center of the robot
        self.parcours = np.zeros((50,50), np.uint8) # occupancy grid
        self.parcours_2_pix=[len(self.image[0])/len(self.parcours[0]),len(self.image)/len(self.parcours)]
        self.objective_red = None
        self.objective_green = None
        self.r_in_pix = 0
        self.r_in_real = 6

The Object Vision is going to initialise all these values and we are going to call update methods and at any time, there are also methods for visualisation and others returning  coefficient values like the conversion from pixel to a real value.


Here is a photo example to illustrate the vision code : 

<img src="Images/Image13.jpeg" width="220"/>

In order to output a global map, each different object has to be detected. In order to detect the robot and its direction we put two blue rounds on it with different sizes, the small one is pointing in its direction. The black objects are the obstacles. The red represents the destination point. The green represents a checkpoint. 

#### Methods in the class

##### resize()

Resize the image returns by the camera so that the computation time is smaller. 

##### read_image()

Read a new image from the camera.

##### mask_thresh (image, thresh_low, thresh_high)

Make a mask of the image in the range specified by the thresholds.

#### redefine_mask

Apply Morphological Operations so that erases noise in the mask.


<img src="Images/Denoise.png" width="220"/>
<img src="Images/Boom.png" width="220"/>

##### get_robot()

Get the position of the robot in the image stored.

Here is a photo of the mask after mask_thresh with black and redefine mask for denoising

![alt text](Images/Mask_circles.png "Title")

Then we detect the circles with cv2 HoughCircle and Compute their centers

![alt text](Images/Image_Robot.png "Title")

##### get_obstacles()

Get the position of the bounding box of the obstacles and returns an occupancy grid for the A* algorithm



Here are photos of the masks after mask_thresh with black and redefine mask for denoising  
![alt text](Images/Mask_obstacles.png "Title")

Then we create a bounding box around each obstacle  
![alt text](Images/Image_obstacles.png "Title")

This parcours is returned as occupancy grid and the obstacles are also dilated so it is easier to dodge them   
<img src="Images/parcours_a_star.png" width="220"/>

##### get_objectives()

Get the center of the objective.

Here is a photo of the mask after mask_thresh with green and red and redefine mask for denoising  
![alt text](Images/Mask_objective_green.png "Title")
![alt text](Images/Mask_objective_red.png "Title")

Then we create a bounding box around each objective and get their center  
![alt text](Images/Image_Objectives.png "Title")


##### print_path()

Print the path generated by the A* algorithm for the path planning on the image.  

<img src="Images/Image50.jpeg" width="220"/>

##### get_first_image_cam()

Crop the image so that it fits the arena and store the coordinates of the arena.
 
It first creates a mask of the arena with mask_thresh and morphological Operations.  
<img src="Images/Image_reboardered.png" width="220"/>

Then we detect the angles of this mask and recompute a reboardered image from the angles

##### get_image_cam()

Use the coordinates of the arena to crop the Image .

##### get_pix_2_real()

Returns a factor to convert from pixel value to real value in cm.

##### get_grid_2_real ()

Returns a factor to convert from grid value to real value in cm.

##### update

Do  read_image(), get_robot(), get_obstacles(), get_objectives(). All the necessary task to detect all the object on the map.


![alt text](Images/Image.png "Title")

In [ ]:
while(True):
    Vision.update()
    cv2.imshow('Show', Vision.image)   
    if cv2.waitKey(1) == ord("q"):
        break
        
Vision.cam.realease()
cv2.destroyAllWindows()

## Global navigation

For the purpose of navigation, we created a clas that represents the robot. The class can be found in utils.py

In [ ]:
# ******** CONSTANTS ********

MOTORSPEED = 50  # Forward speed
MOTOR_ROT = 50  # Rotation speed
FULLROTATIONTIME = 16.5  # time in seconds to turn 2 pi rad


# ******** CLASS ********
class MyThymio(object):
    """
    Class representing our Thymio robot, gathering state information and class
    methods to modifiy its state, command the motors.
    """

    def __init__(self, th, x, y, theta):
        """
        Initialisation of the object
        :param th: instance linking to the Thymio robot connected via USB
               x,y: x and y positions of the center of the Thymio
               theta: the absolute angle orientation of the Thymio
        :return: creates the object of class MyThymio
        """

        self.th = th
        self.x = x
        self.y = y
        self.angle = theta

Here are the methods that we can call on an instance: 
- def get_pos(self)
- def set_pos(self, x, y)
- def motor_stop(self)
- def motor_forward(self)
- def motor_rotate(self, direction)
- def get_angle(self)
- def set_theta(self, theta)
- def get_speed(self)
- def get_previous_pos(self)

### A-Star Path Planning

We use the same A* algorithm provied in the TP. It is based on the pseudocode provided in [Wikipedia](https://en.wikipedia.org/wiki/A*_search_algorithm)  
The vision module is responsible for calculating the occupancy grid, the size of the grid is modular and can be adjusted in the Vision module. For our project, we fixed a size of 50  by 50 for performance reasons. A 50 by 50 grid takes around 4 seconds to be resolved while a grid of 60 by 60 takes 12 seconds (can vary depending on the power of the computer) that is why we opted for that size.  
For a better looking and more smooth navigation, we also opted for 8-connectivity movement (8N)  
**Note:** The implementation of the code can be found in the file A_star_algorithm.py

for the given arena:    
![alt text](report_images/A_star_0.jpeg "Title")    
The vision would generate the following mask:    
![alt text](report_images/A_star_1.png "Title")    
and generate the following grid    
![alt text](report_images/A_star_2.png "Title")  
For the first checkpoint (in green) we generate the following path  
![alt text](report_images/visited_green.png "Title")  
And for the Goal (in red) we obtain the following path:  
![alt text](report_images/visited_red.png "Title")  

The following function was created to directly call the A star algorithm, it only takes as input the Start, the Goal and the occupency grid 

```def run_A_Star(occupancy_grid, start, goal):```

## Filter

### Kalman Filter    
Our Kalman filter consists of a prediction step and a correction step. We use simple odometry to estimate the robot position and the camera to correct the robot position. The filter was also designed to work on linear movements only.  
The global navigation consists of translation steps and rotations steps done seperately, but we will assume that the rotations do not infer any error in the odometry. This choice was made because the camera is not sensitive enough to detect small rotations and would return false values. We also found that using only the translation is sufficient and the filter performs still well. We also use a constant speed of 50 for the Thymio.
The filter takes the position of the robot as well as its orientation as an input vector.  
All the calculus was made in centimeters for the robot positions and in radians for the angles  
**Note:** the implementation of the Kalman filter can be found in the file Kalman.py

$$X = \begin{bmatrix} x\\
y\\
\alpha
\end{bmatrix}$$

We takes an arbitrary covariance matrix with high values as we are uncertain of where the robot starts
$$P = \begin{bmatrix} 100 && 0 && 0\\
0 && 100 && 0\\
0 && 0 && \frac{\pi}{4}
\end{bmatrix}$$  





The odometry uses this dynamic equation:  

$$X_{priori}=\begin{bmatrix} x_{priori}\\
y_{priori}\\
\alpha_{priori}
\end{bmatrix} = \begin{bmatrix} x_{previous}\\
y_{previous}\\
\alpha_{previous}
\end{bmatrix} + 
\begin{bmatrix} speed*\Delta t*sin(\alpha_{previous})\\
speed*\Delta t*cos(\alpha_{previous})\\
0
\end{bmatrix}
$$  

For the computation of the new covariance matrix, we take the jacobian of the dynamic matrix.

$$A = \begin{bmatrix} 1 && 0 && -speed*\Delta t*sin(\alpha_{previous})\\
0 && 1 && -speed*\Delta t*sin(\alpha_{previous})\\
0 && 0 && 0
\end{bmatrix}$$

and we finally add the uncertainty matrix to our covariance matrix. we obtain:

\begin{equation}
P_{priori}=APA^{T} + Q
\end{equation}


For the matrix Q we use the variance of the robot speed to calculate a position variance using sampling, we obtain a value of  $\sigma_x$ = $\sigma_y$ = 0.123 cm. The error on the angle is set arbitrarily to $3^{\circ}$ = $\frac{\pi}{60}$ 

$$Q = \begin{bmatrix} 0.123 && 0 && 0\\
0 && 0.123 && 0\\
0 && 0 && \frac{\pi}{60}
\end{bmatrix}$$

Once the estimation step is complete, we use the update step to correct the robot position using the robot position returned by the camera.  
In this step we simply assume the noise R on the camera readings to be equal to $\sigma$ = 2*pixel size (cm). The value of the pixel size in centimeters is given by the the vision agent. We obtain diffent values of pixel size for the x and y values depending on the size of the Arena we are using. We also approximate the angle error on the camera to $4.5^{\circ}$ = $\frac{\pi}{40}$ 

$$R = \begin{bmatrix} \sigma_x && 0 && 0\\
0 && \sigma_y && 0\\
0 && 0 && \frac{\pi}{40}
\end{bmatrix}$$

The measurement matrix H is a unit diagonal matrix as we simply take the values returned by the vision module without modifications. 

$$H = \begin{bmatrix} 1 && 0 && 0\\
0 && 1 && 0\\
0 && 0 && 1
\end{bmatrix}$$

We then simply calculate the innovation $i$ and the prediction covariance S in order to calculate the Kalman gain K. The equations are as follow

\begin{equation}
i = X_{mesure} - HX_{priori}  
\end{equation}
\begin{equation}
S=HP_{priori}H^{T} + R
\end{equation}

The Kalman K gain tells how much the predictions should be corrected based on the measurements

\begin{equation}
K=P_{priori}H^{T}S^{-1}
\end{equation}

We can then calculate a posterior position estimate and a posterior covariance matrix estimate

\begin{equation}
X_{post} = X_{mesure} + ki  
\end{equation}
\begin{equation}
P_{post}=P_{priori} - KHP_{priori}
\end{equation}

Given that the image quality of the camera isn't very high, we can get errors on the measured values at random times. To make up for that we simply discard the measured values by the camera and only use the estimated values in the Kalman filter. A boolean variable (robot_detected) is simply set to True by the vision module in order for the Kalman filter to use the prediction only 

The final Kalman filter can be called by the function: 

```def kalman_estimate(X_previous,P_previous,delta_t,speed)```

it returns the corrected position and angle of the robot

\begin{equation}
X_{final}= kalman\_estimate(X\_previous,P\_previous,delta\_t,speed)
\end{equation}

## Local navigation using proximity sensors


The robot needs to avoid the obstacles unseen by the vision. To do so, the horizontal proximity sensors of the robot are used to detect if there is anything in front of him. The objective is then to rotate in place until the obstacle is not detected with the sensors then to move from the obstacle by going forward for TIME_FORWARD seconds or until another obstacle is detected.

**Limitations**
- The robot cannot move through an arc of circle but can only move straight forward or turn around the center of its wheels axis. As a result the wall of the obstacle cannot be followed smoothly to find again the optimal path. The only solution to avoid the obstacle is to rotate the robot in place until the obstacle is not detected with the sensors then go forward for a fixed amount of time. This is a limited approach but simple and help us to not lose track of the robot's position by recording the new angle of the robot and the time spent going forward.The angle and the time spent going forward Ts will then be given as input to the function for the kalman filter to estimate the new position of the robot. 
- The fixed amount of time TIME_FORWARD is set experimentally.If it's too long then the robot can run over fixed obstacles seen by the vision but not from the proximity sensors because the obstacles seen by the vision are black rectangles printed on a paper.

In [ ]:
# ******** CONSTANTS ********

THRESHOLD_SIDE = 100  # threshold setting the distance to the wall before going away of it
THRESHOLD_OBST = 2250  # threshold for entering in local navigation
TIME_FORWARD = 2  # time in seconds going away from obstacle before new computation


# ******** FUNCTIONS ********

def obstacle_avoidance(myRobot, prox_sens):
    """
    The robot avoid the obstacle by turning on himself until the obstacle is not detected
    then goes forward
    :param
        myRobot: our robot
        prox_sens: values of the proximity sensors
    :return:
        Ts: time spent going forward
    """
    # The obstacle is on the left - > rotate right
    if (prox_sens[0] + prox_sens[1]) > (prox_sens[4] + prox_sens[3]):
        Ts = bypass("right", prox_sens, myRobot)
    # The obstacle is on the right - > rotate left
    else:
        Ts = bypass("left", prox_sens, myRobot)

    return Ts


def bypass(direction, prox_sens, myRobot):
    """
    Bypass the obstacle by turning on himself until the obstacle is not detected
    then goes forward
    :param
        direction: direction of the rotation
         prox_sens: values of the proximity sensors
        myRobot: our robot
    :return:
        Ts: time spent going forward
    """
    reset_timer = 1  # to start the timer only the first time one enter into the loop
    
    # the robot turn on himself until the sensor don't detect the obstacle
    while sum(prox_sens[i] > THRESHOLD_SIDE for i in range(0, 5)) > 0:

        myRobot.motor_rotate(direction)

        if reset_timer:
            t_start = time.perf_counter()
            reset_timer = 0

        prox_sens = myRobot.th["prox.horizontal"]
        if sum(prox_sens[i] > THRESHOLD_SIDE for i in range(0, 5)) == 0:
            # Stop the timer
            t_stop = time.perf_counter()
            turning_time = t_stop - t_start
            # Computes the angle variation
            dtheta = turning_time * 2 * m.pi / utils.FULLROTATIONTIME
            # Update theta
            if direction == "right":  # Opposite from convention
                theta = myRobot.get_angle() + dtheta
            else:
                theta = myRobot.get_angle() - dtheta

            myRobot.set_theta(theta)

            # Go forward
            Ts = local_forward(myRobot)

    return Ts


def local_forward(myRobot):
    """
    Go forward for TIME_FORWARD seconds or until another obstacle is detected
    :param myRobot: our robot
    :return:
        Ts: time spent going forward
    """
    # Set motors speed to go forward
    myRobot.motor_forward()

    # Increment time counter while checking for obstacles
    step_forward = TIME_FORWARD / 16
    nb_step = 0
    for i in range(16):
        prox = myRobot.th["prox.horizontal"]
        # Break if an obstacle is detected
        if sum(prox[i] > THRESHOLD_SIDE for i in range(0, 5)):
            break
        time.sleep(step_forward)
        nb_step = nb_step + 1

    myRobot.motor_stop()

    # Compute time forward and average speed
    Ts = nb_step * step_forward

    return Ts

## Motor Control

From the actual estimate of position and the set of points from the optimal path, the function **next_node** set the next node to visit and the function **go_to** adjusts the orientation of the robot by rotating him towards his targeted node then sets the motors to go forward.  

*Notes:*

- In order to avoid that the robot correct its orientation too often, the next node tracked is taken a few points (STEP_SKIP) ahead of the actual closest point of the optimal path. The drawback of this approach is that the robot could cross an obstacle instead of stricly respecting a planned corner.
- In the function **go_to**, we only adjust the orientation of the robot towards his targeted node when the difference between the current angle of Thymio the planned one is higher than $\pi$/16 to avoid changing the orientation too often.

In [ ]:
#constants
STEP_SKIP = 2 

def next_node(myRobot, path):
    """
    Set the next node to visit

    :param:
        thymio: our robot
        path: optimal path returned from A* as a list of nodes.
        next node. makes the navigation smoother.
    :return:
        next_node: optimal next node (x,y) to visit
    """

    # Compute distance between Thymio and each position of the optimal path
    dist = np.linalg.norm(myRobot.get_pos() - path , axis=1)
    # Selects the closest point
    step = np.argmin(dist)
    
    if step + STEP_SKIP < len(path):
        _next = path[step + STEP_SKIP]
    else:
        _next = path[-1]
    return _next


def go_to(myRobot, next_pos):
    """
    adjust the orientation of the robot by rotating him towards his targeted node then set the motors to go forward
    :param:
        thymio: our robot
        next_node: targeted node to visit next.
    """
    target_vector = next_pos - myRobot.get_pos()  # vector between actual node and next node.
    theta_ref = m.atan2(target_vector[1], target_vector[0])  # Vector angle from x-axis
    dtheta = utils.normalise_angle(theta_ref - myRobot.get_angle())  # Normalize angle to minimise rotation
    
    # Only rotate when the difference between the current angle of Thymio the planned one is higher than Pi/16 
    if abs(dtheta) > m.pi / 16:
        theta = myRobot.get_angle() + dtheta
        
        # start the rotation
        if dtheta < 0:  # Opposite from convention
            myRobot.motor_rotate("left")
        else:
            myRobot.motor_rotate("right")
            
        # sleep until the rotation is done
        time.sleep(abs(utils.FULLROTATIONTIME * dtheta / (2 * m.pi)))
        myRobot.motor_stop()
        # updata theta
        myRobot.set_theta(theta)
    
    #Go forward
    myRobot.motor_forward()



## Switching between global and local navigation 
We call the function **follow_path** 2 times. The first time, our objective is the first green goal and the second time the objective is the red goal.
While going from a start position to the the designated goal, the robot moves along the global optimal path obtained by the function **run_A_Star**.  

If a local obstacle is detected with the proximity sensors, the robot enters the local navigation , moves to avoid the obstacle locally, calls the function **callKalman** to estimate his new position and then reconstruct the path to the goal using **run_A_Star** again. 

**Limitations**  
- Kidnapping not allowed
- To correctly track the robot, it requires a slow speed and a precise model. For this purpose, it is imposed to move the robot only by turning around the center of its wheels axis or going straight forward.
- This algorithm needs more room to avoid the obstacle. Here the map is too small for the robot size so we can only make one obstacle avoidance locally.

In [ ]:
def follow_path(myRobot, Vision, path, goal):
    """
    Follow the path util the goal is reached and stop
    :param myRobot: our robot
    :param Vision:  our vision agent
    :param path: the path to follow
    :param goal: the goal
    """
    last_state = "global"  # robot state "global" or "local"
    next_pos = path[STEP_SKIP]

    # Adjust angle and goes forward
    go_to(myRobot, next_pos)
    # Starts timer
    t_start = time.perf_counter()
    # Keep going while goal not reached
    while np.linalg.norm(myRobot.get_pos() - goal) > EPS_GOAL:

        prox_sens = myRobot.th["prox.horizontal"]

        # Checking if should be in local or global navigation
        if sum([prox_sens[i] > local_navigation.THRESHOLD_OBST for i in range(0, 5)]) < 1:
            state = "global"  # global navigation

        else:
            state = "local"  # obstacle avoidance

        # Global navigation
        if state == "global":

            if last_state == "global":
                t_stop = time.perf_counter()
                Ts = t_stop - t_start
                # estimate new position
                callKalman(myRobot, Vision, Ts)

            next_pos = next_node(myRobot, path)

            # adjusts the orientation then goes forward
            go_to(myRobot, next_pos)

            # Restarts timer
            t_start = time.perf_counter()
            
            last_state = "global"

        # Local navigation
        elif state == "local":
            # Stops
            myRobot.motor_stop()

            # If previously in global navigation, computes actual position
            if last_state == "global":
                # Records time forward in global before entering in local
                t_stop = time.perf_counter()
                Ts = t_stop - t_start
                # estimate new position
                callKalman(myRobot, Vision, Ts)

            # Enters local navigation
            Ts = local_navigation.obstacle_avoidance(myRobot, prox_sens)

            # estimate new position
            callKalman(myRobot, Vision,Ts)

            # Reconstruct the path to the goal
            path, visitedNodes = run_A_Star(Vision.parcours, myRobot.get_pos(), goal)

            last_state = "local"

    print("Goal reached")
    myRobot.motor_stop()

## Connexion to Thymio 

In [ ]:
from Thymio import Thymio
#With cable
th = Thymio.serial(port="COM7", refreshing_rate=0.1)
#Wireless
#th = Thymio.serial(port="\\.\COM5", refreshing_rate=0.1)

## Implementation


In [ ]:
# Get map information and robot initial pose
Vision.update()
cv2.imwrite("image_start.png",Vision.image)
goal=Vision.objective_green
path, visitedNodes = run_A_Star(Vision.parcours, Vision.center_robot, goal)
Vision.path=path
Vision.print_path()
x = Vision.center_robot[0]
y = Vision.center_robot[1]
theta = utils.deg_to_rad(Vision.angle)
#Initialise the robot
myRobot=utils.MyThymio(th,x,y,theta)
th.set_var("motor.right.target",0)
th.set_var("motor.left.target",0)
#Initialise Kalman
Kalman.kalman_init(Vision)

# Go to first goal
navigation.follow_path(myRobot,Vision,path,goal)

goal=Vision.objective_red
path, visitedNodes = run_A_Star(Vision.parcours, myRobot.get_pos(), goal)
Vision.path=path

# Go to final goal
navigation.follow_path(myRobot,Vision,path,goal)

cv2.imwrite("image_end.png",Vision.image)
Vision.cam.release()